In [14]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
from juliacall import Main as jl
def load_module(module: str):
    from juliacall import Main as jl
    mod_aliases = {
        "DataLoading": "DAT",
        "Reporting": "REP",
        "Processing": "BPR",
        "CLI": "CLI"
    }

    if module not in mod_aliases:
        raise ValueError(f"Module {module} not found. Please choose from {mod_aliases.keys()}")

    module = mod_aliases[module]
    return jl.seval(f"import Baysor; Baysor.load_module(Baysor.{module})")
    from juliacall import Main as jl
bpr = load_module("Processing")
jl.seval("using SparseArrays;")
jl.seval("using DataFrames;")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [15]:
test_data_path =  "/home/viktor_petukhov/mh/spatial/CellSegmentation/data/merfish_moffit/merfish_coords_adj.csv"
df_spatial = pd.read_csv(test_data_path)
df_spatial.gene = pd.factorize(df_spatial.gene)[0]
df_spatial_test = df_spatial[:5000].copy()
clusters = 3

In [16]:
from molecule_clustering import estimate_molecule_clusters
from noise_estimation import estimate_confidence

In [22]:
mean_dists, confidence, min_confidence = estimate_confidence(df_spatial_test, 15)

knn


  0%|                                                                                                                                                                    | 4/10000 [00:00<01:19, 125.29it/s]

-0.001957821902669905
Converged...


In [17]:
df_spatial_test_jl = df_spatial_test
df_spatial_test_jl.gene += 1 
df_spatial_test_jl = jl.DataFrame(df_spatial_test_jl)

In [26]:
mean_dists_jl, confidence_jl, d1, d2 = bpr.estimate_confidence(df_spatial_test_jl, nn_id=15)

[69.13211988648982, 147.85127662620977, 49.39635614091387, 47.885279575251516, 78.6956161421969]


In [34]:
print('mean squared error in confidence between python & julia:')
print(np.mean((np.array(confidence) - np.array(confidence_jl)**2)))

mean squared error in confidence between python & julia:
0.06511384199841755


In [45]:
df_spatial_test['confidence'] = np.array(confidence)
df_spatial_test_jl = jl.DataFrame(df_spatial_test)

In [60]:
%%time
res = estimate_molecule_clusters(df_spatial_test, 3)

Clustering molecules...
knn


  0%|▊                                                                                                                                                                   | 47/10000 [00:06<21:34,  7.69it/s]


Done
CPU times: user 49 s, sys: 1.01 s, total: 50 s
Wall time: 6.93 s


In [61]:
res['assignment'][:350]

array([1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 2, 0, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 0, 0, 0,
       2, 2, 2, 0, 0, 2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 2, 2, 0, 2, 0, 0,
       2, 0, 2, 0, 0, 2, 0, 2, 2, 0, 2, 2, 0, 2, 0, 0, 0, 0, 0, 2, 2, 0,
       2, 2, 0, 0, 0, 2, 2, 0, 2, 2, 2, 0, 0, 2, 0, 2, 2, 0, 0, 2, 2, 2,
       2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 2, 0, 0, 0, 2,
       2, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2, 0, 0,
       2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 0, 0,
       0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2,
       0, 2, 2, 2, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 0, 0,
       0, 2, 0, 2, 2, 2, 2, 0, 0, 0, 2, 2, 2, 2, 2,

In [59]:
res_jl = bpr.estimate_molecule_clusters(df_spatial_test_jl, 3)

[ Info: Clustering molecules...
[ Info: Algorithm stopped after 65 iterations. Error: 1.41e-5. Converged: true.
[ Info: Done


In [55]:
np.array(res_jl.assignment)[:350]

array([2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 1, 3, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 1, 1, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 3,
       3, 3, 3, 3, 1, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1,
       3, 1, 3, 1, 1, 3, 1, 3, 3, 1, 3, 3, 1, 3, 1, 1, 1, 1, 1, 3, 3, 1,
       3, 3, 1, 1, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 1, 3, 1, 1, 3, 1, 1, 1, 1, 1, 1, 1, 3, 3, 3, 1, 1, 1, 3,
       3, 1, 3, 3, 3, 3, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1,
       3, 1, 3, 3, 3, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 1, 1, 3, 3, 3, 3, 3, 3,
       1, 3, 3, 3, 3, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 1, 3, 3, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3,

In [62]:
(res['assignment'] == (np.array(res_jl.assignment) - 1)).mean()

0.899